# Data Preparation 
## 1. Reading Archive Data Group

This datagroup consist timestamp observations of sismic activity and educational socioeconomic data.

In [1]:
import pandas as pd
import numpy as np
import geopandas as gdp


In [2]:
# defining the archive data path that we will need to read several datasets.
path = '../data/archive/'

#make a list of the name of the files we will need to read.in order not to read all of them seperately
lista = ['Earthquakes','Earthquakes2year', 'eClass','eNumber',
     'eStudent', 'eTeachers','hClass','hNumber', 'hStudents',
    'hTeachers','hTypes','ilcelervekoordinatlar', 'mClass',
    'mNumber', 'mNumber','mStudents', 'mTeachers', 'pClass',
    'pNumber', 'pStudents', 'pTeachers',]
# making a dictionary of dataframe.
data = {}
# reading the files and adding them to the dictionary.
for file_name in lista:
    df = pd.read_csv(f"{path}{file_name}.csv")
    data[file_name] = df

This is a  dataset from https://www.kaggle.com/datasets/ardaorcun/turkey-6-february-disaster-and-related-datas which is a combination of several datasets from other sources below.

For earthquakes: https://deprem.afad.gov.tr/event-catalog

For any data about population and economic numbers: https://www.tuik.gov.tr/

For datas about education: https://istatistik.meb.gov.tr/

Earthquakes dataset represents the sismic activity of the region. all activities are listed with a timestamp.

ilcelervkordinatlar.csv has the population data from https://www.tuik.gov.tr/ 

The other csv files has the dataset about education according to prefix:

    datasets starting with e =>  elementary school data
    datasets starting with p => preschool data
    datasets starting with h => high school data
    datasets starting with m => middle school data

according to suffixes:

    Class => number of classes avaliable
    Number => number of institutions available
    Students => number of students
    Teachers = > number of teachers
    Type => for high schools type of the highschool

In [23]:
def rename_columns(df, prefix):
    return df.rename(columns={
        'Resmi': f'{prefix}_public',
        'Özel': f'{prefix}_private',
        'Toplam': f'{prefix}_total',
        'R Erkek': f'{prefix}_male_public',
        'R Kadın': f'{prefix}_female_public',
        'R Toplam': f'{prefix}_total_public',
        'Ö Erkek': f'{prefix}_male_private',
        'Ö Kadın': f'{prefix}_female_private',
        'Ö Toplam': f'{prefix}_total_private',
    })

def rename_hs_type(df, type_name, prefix):
    df_filtered = df[df['Okul Türü'] == type_name]
    return rename_columns(df_filtered, prefix)


In [24]:

# Elementary schools
eClass = rename_columns(data['eClass'], 'eclass')
eNumber = rename_columns(data['eNumber'], 'enumber')
eStudent = rename_columns(data['eStudent'], 'e')
eTeachers = rename_columns(data['eTeachers'], 'et')

# High schools
hClass = rename_columns(data['hClass'], 'hclass')
hNumber = rename_columns(data['hNumber'], 'hnumber')
hStudents = rename_columns(data['hStudents'], 'h')
hTeachers = rename_columns(data['hTeachers'], 'ht')

# High school types
hType_din = rename_hs_type(data['hTypes'], 'Din Öğretimi', 'hs_religious')
hTypes_meslek = rename_hs_type(data['hTypes'], 'Mesleki ve Teknik Ortaöğretim', 'h_occupational')
hTypes_genel = rename_hs_type(data['hTypes'], 'Genel Ortaöğretim', 'h_normal')

# Middle schools
mClass = rename_columns(data['mClass'], 'mclass')
mNumber = rename_columns(data['mNumber'], 'mnumber')
mStudents = rename_columns(data['mStudents'], 'm')
mTeachers = rename_columns(data['mTeachers'], 'mt')

# Preschools
pClass = rename_columns(data['pClass'], 'pclass')
pNumber = rename_columns(data['pNumber'], 'pnumber')
pStudents = rename_columns(data['pStudents'], 'p')
pTeachers = rename_columns(data['pTeachers'], 'pt')

In [25]:
data

{'Earthquakes':        Unnamed: 0                Date  Longitude  Latitude  Depth   Rms Type  \
 15816       15816 2023-02-05 00:00:55     38.828    38.255   6.73  0.16   ML   
 14453       14453 2023-02-05 00:54:55     43.954    41.209  11.26  0.29   ML   
 12787       12787 2023-02-05 02:31:06     44.942    38.648  11.86  0.58   ML   
 6374         6374 2023-02-05 02:37:25     42.641    38.389   7.12  0.28   ML   
 9837         9837 2023-02-05 03:18:37     39.186    38.469   7.00  0.28   ML   
 ...           ...                 ...        ...       ...    ...   ...  ...   
 574           574 2023-03-10 07:58:34     37.493    37.985   7.01  0.41   MW   
 6268         6268 2023-03-10 08:02:35     38.192    38.054   7.00  0.66   ML   
 12575       12575 2023-03-10 08:04:56     37.387    37.951   7.00  0.91   ML   
 11718       11718 2023-03-10 08:08:34     36.199    37.805   7.00  0.67   ML   
 15015       15015 2023-03-10 08:14:17     36.939    37.289  13.04  0.32   ML   
 
        Mag

School datasets has the same column name in order not to mix with each other we are changing the column names. "Resmi" means public and "özel" means private. "erkek" and "kadin" mean male and female students. 

In [8]:

# Now we are merging the dataframes according on Province column 
# List of dataframe names to be merged

df_names = [eClass, eNumber, eStudent, eTeachers, hClass, hNumber,
            hStudents,hType_din, hTypes_meslek, hTypes_genel,
            hTeachers,mClass, mNumber, mStudents,
            mTeachers, pClass, pNumber, pStudents, pTeachers]

# Initialize the merged dataframe with the first dataframe in the list
merged = eClass

# Loop over the remaining dataframes and merge them into the merged dataframe
for df_name in df_names[1:]:
    
    merged = pd.merge(merged, df_name, on='Şehir')

# View the resulting merged dataframe
merged= merged.rename(columns={'Şehir':'Province'})
merged.columns

merged.to_csv('../data/archive/merged.csv', index=False)

merged


,Province,eclass_public,eclass_private,eclass_total,enumber_public,enumber_private,enumber_total,e_male_public,e_female_public,e_total_public,...,p_female_private,p_total_private,p_total,pt_male_public,pt_female_public,pt_total_public,pt_male_private,pt_female_private,pt_total_private,pt_total
0,ADANA,1456,744,2200,638,150,788,22426,21309,43735,...,3557,7772,51507,203,2121,2324,14,543,557,2881
1,ADIYAMAN,579,180,759,375,35,410,8659,8078,16737,...,583,1349,18086,121,647,768,7,131,138,906
2,DİYARBAKIR,1686,256,1942,1047,70,1117,25021,23239,48260,...,1509,3274,51534,341,1872,2213,6,220,226,2439
3,GAZİANTEP,1657,522,2179,675,93,768,28937,27087,56024,...,1881,4063,60087,214,2123,2337,7,389,396,2733
4,HATAY,1396,628,2024,683,128,811,23986,22570,46556,...,2299,5137,51693,195,1841,2036,12,405,417,2453
5,MALATYA,558,246,804,310,53,363,6996,6577,13573,...,1093,2281,15854,83,709,792,9,87,96,888
6,KAHRAMANMARAŞ,876,204,1080,479,53,532,14232,13363,27595,...,973,2121,29716,158,1031,1189,7,135,142,1331
7,ŞANLIURFA,1977,273,2250,1307,77,1384,32861,31207,64068,...,1146,2513,66581,345,2152,2497,11,217,228,2725
8,KİLİS,173,33,206,96,10,106,2768,2727,5495,...,150,321,5816,47,205,252,0,50,50,302
9,OSMANİYE,422,123,545,226,36,262,6799,6302,13101,...,653,1356,14457,71,593,664,8,115,123,787


In [9]:
# saving the dataframe as pickle file to be called later in the notebook and not to lose the manipulations.
merged.to_pickle('../data/archive/merged.pkl')

In [10]:
city = data['ilcelervekoordinatlar']
city

,Unnamed: 0,İlçeler,Kayıtlı Nüfus,Longitude,Latitude
0,0,Adana(Aladağ)-1757,5642,35.3950,37.5467
1,1,Adana(Ceyhan)-1219,44498,35.8147,37.0286
2,2,Adana(Çukurova)-2033,120591,35.2721,37.0400
3,3,Adana(Feke)-1329,6110,35.9109,37.8182
4,4,Adana(İmamoğlu)-1806,8464,35.6582,37.2596
...,...,...,...,...,...
108,108,Şanlıurfa(Hilvan)-1393,8935,38.9552,37.5852
109,109,Şanlıurfa(Karaköprü)-2093,65067,38.7992,37.2127
110,110,Şanlıurfa(Siverek)-1630,52063,39.3168,37.7546
111,111,Şanlıurfa(Suruç)-1643,20463,38.4243,36.9751


In [11]:
import pandas as pd

# Assume city is your DataFrame and 'İlçeler' is your column

# Step 1: Get rid off last 5 characters
city['İlçeler'] = city['İlçeler'].str[:-5]

# Step 2: Uppercase all
city['İlçeler'] = city['İlçeler'].str.upper()

# Step 3: Separate into 'province' and 'city' columns
city['Province'], city['city'] = city['İlçeler'].str.split('(', 1).str
city['city'] = city['city'].str.rstrip(')')  # remove trailing ')'





/tmp/ipykernel_68944/3724562047.py:12: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  city['Province'], city['city'] = city['İlçeler'].str.split('(', 1).str
/tmp/ipykernel_68944/3724562047.py:12: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  city['Province'], city['city'] = city['İlçeler'].str.split('(', 1).str


The next dataset has locations of the towns and provinces and population

In [12]:
merged_data = city.merge(merged, on='Province')

merged_data.to_pickle('../data/manipulated/merged_data.pkl')

In [13]:
from shapely.geometry import Point 

merged_data = pd.read_pickle('../data/manipulated/merged_data.pkl')


merged_data['Geometry'] = merged_data.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)

merged_data.to_pickle('../data/manipulated/merged_data.pkl')

In [14]:
merged_data

,Unnamed: 0,İlçeler,Kayıtlı Nüfus,Longitude,Latitude,Province,city,eclass_public,eclass_private,eclass_total,...,p_total_private,p_total,pt_male_public,pt_female_public,pt_total_public,pt_male_private,pt_female_private,pt_total_private,pt_total,Geometry
0,0,ADANA(ALADAĞ),5642,35.3950,37.5467,ADANA,ALADAĞ,1456,744,2200,...,7772,51507,203,2121,2324,14,543,557,2881,POINT (35.395 37.5467)
1,1,ADANA(CEYHAN),44498,35.8147,37.0286,ADANA,CEYHAN,1456,744,2200,...,7772,51507,203,2121,2324,14,543,557,2881,POINT (35.8147 37.0286)
2,2,ADANA(ÇUKUROVA),120591,35.2721,37.0400,ADANA,ÇUKUROVA,1456,744,2200,...,7772,51507,203,2121,2324,14,543,557,2881,POINT (35.2721 37.04)
3,3,ADANA(FEKE),6110,35.9109,37.8182,ADANA,FEKE,1456,744,2200,...,7772,51507,203,2121,2324,14,543,557,2881,POINT (35.9109 37.8182)
4,4,ADANA(İMAMOĞLU),8464,35.6582,37.2596,ADANA,İMAMOĞLU,1456,744,2200,...,7772,51507,203,2121,2324,14,543,557,2881,POINT (35.6582 37.2596)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,108,ŞANLIURFA(HILVAN),8935,38.9552,37.5852,ŞANLIURFA,HILVAN,1977,273,2250,...,2513,66581,345,2152,2497,11,217,228,2725,POINT (38.9552 37.5852)
72,109,ŞANLIURFA(KARAKÖPRÜ),65067,38.7992,37.2127,ŞANLIURFA,KARAKÖPRÜ,1977,273,2250,...,2513,66581,345,2152,2497,11,217,228,2725,POINT (38.7992 37.2127)
73,110,ŞANLIURFA(SIVEREK),52063,39.3168,37.7546,ŞANLIURFA,SIVEREK,1977,273,2250,...,2513,66581,345,2152,2497,11,217,228,2725,POINT (39.3168 37.7546)
74,111,ŞANLIURFA(SURUÇ),20463,38.4243,36.9751,ŞANLIURFA,SURUÇ,1977,273,2250,...,2513,66581,345,2152,2497,11,217,228,2725,POINT (38.4243 36.9751)


Earthquakes dataset has the data of measured earthquakes from 5th of February(preivous day of disaster) to 10th of March from  https://deprem.afad.gov.tr/event-catalog 

Dataset consist of below variables:

    Date => timestamp of the sismic activity
    Longitude => longitude
    Latitude => latitude
    Depth => depth 
    Rms => Root mean square
    Magnitude => the measurements
    Location => Province and time name
    Type => ML as local magnitude MW as moment magnitude
    EventID => event identifier of the database.



In [15]:
#extracting earthquake data from data dictionary
earthquake = data['Earthquakes']
# changing Location to uppercase to match with the merged_data
earthquake.Location = earthquake.Location.str.upper()
# changing the date column to datetime format
earthquake.Date = pd.to_datetime(earthquake.Date)
# sorting the values by date
earthquake.sort_values(by='Date', inplace=True)
earthquake

,Unnamed: 0,Date,Longitude,Latitude,Depth,Rms,Type,Magnitude,Location,EventID
15816,15816,2023-02-05 00:00:55,38.828,38.255,6.73,0.16,ML,0.8,PÜTÜRGE (MALATYA),543347
14453,14453,2023-02-05 00:54:55,43.954,41.209,11.26,0.29,ML,1.3,"NINOTSMINDA, SAMTSKHE-JAVAKHETI (GÜRCISTAN) - ...",543353
12787,12787,2023-02-05 02:31:06,44.942,38.648,11.86,0.58,ML,1.6,"KHOY, WEST AZARBAIJAN (İRAN) - [55.57 KM] BAŞK...",543359
6374,6374,2023-02-05 02:37:25,42.641,38.389,7.12,0.28,ML,2.3,TATVAN (BITLIS),543358
9837,9837,2023-02-05 03:18:37,39.186,38.469,7.00,0.28,ML,1.9,SIVRICE (ELAZIĞ),543363
...,...,...,...,...,...,...,...,...,...,...
574,574,2023-03-10 07:58:34,37.493,37.985,7.01,0.41,MW,3.9,NURHAK (KAHRAMANMARAŞ),560818
6268,6268,2023-03-10 08:02:35,38.192,38.054,7.00,0.66,ML,2.3,ÇELIKHAN (ADIYAMAN),560841
12575,12575,2023-03-10 08:04:56,37.387,37.951,7.00,0.91,ML,1.6,NURHAK (KAHRAMANMARAŞ),560838
11718,11718,2023-03-10 08:08:34,36.199,37.805,7.00,0.67,ML,1.7,SAIMBEYLI (ADANA),560839


In [16]:
# Deescriptive statistics of the earthquake data
earthquake.describe()

,Unnamed: 0,Longitude,Latitude,Depth,Rms,Magnitude,EventID
count,15901.00000,15901.000000,15901.000000,15901.000000,15901.000000,15901.000000,15901.000000
mean,7950.00000,37.473933,37.810726,7.684039,0.443839,2.265606,552010.119364
std,4590.36765,1.216845,0.726524,2.404630,0.256377,0.782173,5029.779034
min,0.00000,35.556000,33.835000,0.000000,0.010000,0.300000,543347.000000
25%,3975.00000,36.621000,37.420000,7.000000,0.270000,1.700000,547646.000000
50%,7950.00000,37.179000,37.955000,7.000000,0.420000,2.100000,551940.000000
75%,11925.00000,38.103000,38.137000,7.120000,0.590000,2.700000,556350.000000
max,15900.00000,45.419000,42.497000,43.160000,7.540000,7.700000,560849.000000


In [17]:
import folium

# Create a folium map centered at a specific location
m = folium.Map(location=[37, 37], zoom_start=10)

# Iterate over each row in the merged_data dataframe
for index, row in merged_data.iterrows():
    # Extract the geometry point from the 'Geometry' column
    geometry = row['Geometry']
    
    # Extract the latitude and longitude from the geometry point
    lat = geometry.y
    lon = geometry.x
    
    # Create a marker at the latitude and longitude coordinates
    folium.Marker([lat, lon]).add_to(m)

# Display the map
m


In [18]:
#manipulating the earthquake data 
#dropping the unnamed column
#earthquake.drop(columns= ['Unnamed: 0'], inplace=True)
# extracting Province and Municipality from Location column
earthquake['Province'] = earthquake['Location'].str.extract(r'\((.*?)\)')
earthquake['Municipio'] = earthquake['Location'].str.extract(r'^(.*?)\s*\(')


earthquake['Location_name'] = earthquake['Province']+ "-"+ earthquake['Municipio']

earthquake['Location_name'] 

15816                              MALATYA-PÜTÜRGE
14453    GÜRCISTAN-NINOTSMINDA, SAMTSKHE-JAVAKHETI
12787                   İRAN-KHOY, WEST AZARBAIJAN
6374                                 BITLIS-TATVAN
9837                                ELAZIĞ-SIVRICE
                           ...                    
574                           KAHRAMANMARAŞ-NURHAK
6268                             ADIYAMAN-ÇELIKHAN
12575                         KAHRAMANMARAŞ-NURHAK
11718                              ADANA-SAIMBEYLI
15015                            GAZIANTEP-NURDAĞI
Name: Location_name, Length: 15901, dtype: object

In [19]:
# grouping earthquake data by province to see how many earthquakes happened in each province
earthquake.groupby('Location_name').count()


,Unnamed: 0,Date,Longitude,Latitude,Depth,Rms,Type,Magnitude,Location,EventID,Province,Municipio
Location_name,,,,,,,,,,,,
ADANA-AKDENIZ - İSKENDERUN KÖRFEZI - [09.27 KM] YUMURTALIK,1,1,1,1,1,1,1,1,1,1,1,1
ADANA-ALADAĞ,2,2,2,2,2,2,2,2,2,2,2,2
ADANA-CEYHAN,17,17,17,17,17,17,17,17,17,17,17,17
ADANA-FEKE,12,12,12,12,12,12,12,12,12,12,12,12
ADANA-KOZAN,26,26,26,26,26,26,26,26,26,26,26,26
...,...,...,...,...,...,...,...,...,...,...,...,...
ŞANLIURFA-SIVEREK,2,2,2,2,2,2,2,2,2,2,2,2
ŞANLIURFA-SURUÇ,7,7,7,7,7,7,7,7,7,7,7,7
ŞIRNAK-MERKEZ,1,1,1,1,1,1,1,1,1,1,1,1


From earthquake data we will need earthquake counts and distance to the important earthquakes.

In [20]:
# filtering city data to another frame and saving it as pickle file
city_earthquaques = earthquake.groupby('Location_name').count().reset_index()
#filtering the columns, count and locaation
city_earthquaques = city_earthquaques[['Location_name', 'EventID']]
#renaming the count column
city_earthquaques.rename(columns={'EventID':'Count_lastmonth'}, inplace=True)
#saving the dataframe as pickle file
city_earthquaques.to_pickle('../data/manipulated/city_earthquaques.pkl')

In [21]:

# filtering the most important observations to another dataframe later to be used to check distances.
day_eq= earthquake[earthquake['Magnitude'] > 6.5]
#saving the dataframe as pickle file
day_eq.to_pickle('../data/manipulated/day_eq.pkl')
